In [2]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Okt
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora 
from gensim import models
import gensim

In [3]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [4]:
data =[]
for i in range(0,1):
    txt1 = open('세종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '세종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '백성' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'세종실록'))

In [5]:
# 시리즈 만들기
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("세종실록"," "))
df=df.map(lambda x : x.replace("세종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [6]:
len(df)

2624

In [7]:
okt = Okt()

In [8]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','모두','하니','어찌'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [9]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [10]:
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [11]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	백성	0.012778542004525661
	생각	0.007151797413825989
	하나	0.0071101258508861065
	이제	0.00660011125728488
	비록	0.006322056520730257
	자가	0.006025754380971193
	제사	0.0059900833293795586
	다시	0.005716984625905752
	의논	0.005523922853171825
	정통	0.00523762684315443


Topic ID: 1
	백성	0.02403736487030983
	의창	0.01358215045183897
	흉년	0.010494690388441086
	수령	0.01042970921844244
	곡식	0.009119493886828423
	의논	0.007614972069859505
	정통	0.007492970675230026
	생각	0.006905258633196354
	자가	0.006682756822556257
	비록	0.00654052896425128


Topic ID: 2
	백성	0.013396762311458588
	비록	0.009281869977712631
	생각	0.008895492181181908
	반드시	0.008876023814082146
	만약	0.006906529422849417
	우리	0.0065123094245791435
	고을	0.006018850486725569
	마음	0.005562332924455404
	군사	0.005282407160848379
	도적	0.004750873893499374


Topic ID: 3
	우리	0.015521401539444923
	백성	0.011771949008107185
	생각	0.010701538063585758
	가지	0.0068548996932804585
	마음	0.00623656390234828
	범찰	0.0059411413967609406
	비록	0.00578882545232772

In [12]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.031006 -0.053344       1        1  18.761164
8     -0.063343 -0.028058       2        1  13.170181
9     -0.093975  0.052327       3        1  10.661488
3      0.032256 -0.019209       4        1   9.614742
5     -0.057316 -0.026863       5        1   8.989223
0     -0.006102 -0.023263       6        1   6.853406
2     -0.021394  0.014674       7        1   5.106237
1     -0.076008 -0.000132       8        1   3.599758
16     0.042348 -0.078391       9        1   3.438258
19     0.037171 -0.039420      10        1   3.021709
13    -0.039771 -0.004162      11        1   2.523945
12    -0.030853  0.053723      12        1   2.493418
4     -0.006858  0.011260      13        1   2.428753
6      0.039739 -0.013865      14        1   2.414716
14     0.077632 -0.002034      15        1   1.674377
17    -0.011896  0.032185      16        1   1.667321
18     0.081856  0.080805      17        1   1.664062
15     0.028895  0.029876      18        1   0.927006
10     0.049842  0.017666      19        1   0.601433
7      0.048783 -0.003774      20        1   0.388809, topic_info=     Term         Freq        Total Category  logprob  loglift
10     백성  5905.000000  5905.000000  Default  30.0000  30.0000
424    우리  2052.000000  2052.000000  Default  29.0000  29.0000
55     군사  1470.000000  1470.000000  Default  28.0000  28.0000
12     생각  2194.000000  2194.000000  Default  27.0000  27.0000
11     비록  2257.000000  2257.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
358    가지     3.005067  1437.287109  Topic20  -6.0595  -0.6204
1587   하소     2.832601   778.079407  Topic20  -6.1186  -0.0658
671   의정부     2.808621   757.215210  Topic20  -6.1271  -0.0471
55     군사     2.823630  1470.410767  Topic20  -6.1218  -0.7054
49     고을     2.774235  1543.959351  Topic20  -6.1394  -0.7719

[1565 rows x 6 columns], token_table=       Topic      Freq Term
term                       
644        2  0.812762   가구
358        1  0.244906   가지
358        2  0.146804   가지
358        3  0.065401   가지
358        4  0.151675   가지
...      ...       ...  ...
15889      4  0.027420   희미
15889      5  0.548400   희미
15889     12  0.027420   희미
15889     14  0.329040   희미
15889     17  0.027420   희미

[6445 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 9, 10, 4, 6, 1, 3, 2, 17, 20, 14, 13, 5, 7, 15, 18, 19, 16, 11, 8])

In [13]:
# 의논

In [14]:
data =[]
for i in range(0,1):
    txt1 = open('세종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '세종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '의논' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'세종실록'))

In [15]:
len(df)

2624

In [16]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("세종실록"," "))
df=df.map(lambda x : x.replace("세종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [18]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','어찌','만약'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [19]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [20]:
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [21]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	모두	0.011723233386874199
	하니	0.011701345443725586
	우리	0.009165161289274693
	의논	0.007226805202662945
	영상	0.006126666907221079
	비록	0.004152934532612562
	반드시	0.0037319299299269915
	위판	0.003591692540794611
	중국	0.003385737771168351
	마음	0.0032899705693125725


Topic ID: 1
	의논	0.0212861318141222
	하니	0.011442837305366993
	모두	0.008652149699628353
	화포	0.007949193939566612
	노비	0.0067384024150669575
	천인	0.004965956788510084
	정통	0.0046166046522557735
	의정부	0.004396457690745592
	득종	0.004264162853360176
	그대로	0.004198345821350813


Topic ID: 2
	의논	0.023053202778100967
	하니	0.017720744013786316
	모두	0.009773561730980873
	의정부	0.007916491478681564
	비록	0.007810864597558975
	정통	0.006739513017237186
	이제	0.006266114767640829
	다시	0.005383902229368687
	하소	0.005222133360803127
	설치	0.00472768722102046


Topic ID: 3
	의논	0.014681796543300152
	모두	0.010205459780991077
	하니	0.009852780029177666
	비록	0.00921640358865261
	다시	0.00826715212315321
	반드시	0.007132593542337418
	우리	0.006970818154

In [22]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.036173 -0.023783       1        1  16.040180
3     -0.053766 -0.027024       2        1  11.151448
2     -0.017958 -0.055008       3        1  10.885670
9     -0.077582  0.032560       4        1   7.265937
6      0.029337 -0.058038       5        1   7.021231
10    -0.080090  0.043279       6        1   6.920366
18     0.007989 -0.054656       7        1   6.381405
11    -0.013088 -0.023052       8        1   5.758761
4     -0.025343  0.012600       9        1   5.027966
5     -0.005532 -0.036936      10        1   4.540200
15     0.015119  0.005098      11        1   4.457550
14    -0.009784 -0.023765      12        1   3.076234
12     0.016350  0.023694      13        1   2.855234
8      0.038519 -0.013406      14        1   1.706236
13    -0.001589  0.055309      15        1   1.692775
16    -0.028973  0.051760      16        1   1.629165
1      0.046304  0.007382      17        1   1.374179
7      0.062329 -0.003845      18        1   1.253855
0      0.061110  0.038214      19        1   0.563010
19     0.072823  0.049617      20        1   0.398600, topic_info=     Term         Freq        Total Category  logprob  loglift
8      의논  5615.000000  5615.000000  Default  30.0000  30.0000
124    하니  4420.000000  4420.000000  Default  29.0000  29.0000
54     모두  3798.000000  3798.000000  Default  28.0000  28.0000
3280   범찰   286.000000   286.000000  Default  27.0000  27.0000
92     우리  2129.000000  2129.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
41     다만     4.354021  1221.537842  Topic20  -5.8199  -0.1118
254    마음     4.447448  1425.616943  Topic20  -5.7987  -0.2451
156    서로     3.772724  1052.830200  Topic20  -5.9632  -0.1065
107    자가     3.777112  1537.227539  Topic20  -5.9620  -0.4838
96    의정부     3.423095  1076.502441  Topic20  -6.0605  -0.2260

[1577 rows x 6 columns], token_table=       Topic      Freq Term
term                       
4427       7  0.818290   가계
14480      2  0.134699   가인
14480     12  0.808191   가인
8907       1  0.847791  가장이
18         1  0.160114   가지
...      ...       ...  ...
2166      12  0.026110   흉노
2166      16  0.026110   흉노
3648       7  0.721149   흠절
11861      4  0.850842   흠휼
10667      9  0.951291  흥안군

[6274 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 4, 3, 10, 7, 11, 19, 12, 5, 6, 16, 15, 13, 9, 14, 17, 2, 8, 1, 20])

In [23]:
# 학문

In [24]:
data =[]
for i in range(0,1):
    txt1 = open('세종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '세종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '학문' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'세종실록'))

In [25]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("세종실록"," "))
df=df.map(lambda x : x.replace("세종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [26]:
len(df)

277

In [27]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','모두','비록','어찌','이제'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [28]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [29]:
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

In [30]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	생각	0.008993742987513542
	마음	0.006527609657496214
	반드시	0.005253070965409279
	학문	0.005210165400058031
	우리	0.004708932712674141
	가지	0.004539811983704567
	하니	0.004266179632395506
	아들	0.0037404836621135473
	더욱	0.0033386044669896364
	백성	0.0031491992995142937


Topic ID: 1
	생각	0.006484422367066145
	나이	0.005657383240759373
	학문	0.005609494633972645
	하니	0.005443725269287825
	마음	0.005389419384300709
	반드시	0.004755795933306217
	백성	0.004714625887572765
	다시	0.00385461887344718
	제사	0.0035175506491214037
	만약	0.0033831733744591475


Topic ID: 2
	우리	0.009201796725392342
	마음	0.007967965677380562
	하니	0.007136222906410694
	생각	0.007127257063984871
	반드시	0.005890964064747095
	가지	0.005741041619330645
	백성	0.004327032249420881
	더욱	0.004159070551395416
	학문	0.004064911510795355
	경은	0.003951216582208872


Topic ID: 3
	학문	0.011597900651395321
	하니	0.007428581826388836
	도회	0.005251562222838402
	백성	0.004853981547057629
	자가	0.004681865219026804
	가지	0.004098205827176571
	마음	0.004001152

In [31]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.003505  0.023263       1        1  13.419729
7     -0.016727  0.006851       2        1  11.682032
9     -0.017948 -0.019101       3        1   8.111159
10    -0.007523  0.020673       4        1   7.267716
14    -0.014388  0.008542       5        1   6.657263
13     0.014294 -0.045706       6        1   6.594659
11     0.006958  0.007241       7        1   5.582543
0      0.001977  0.009467       8        1   5.192235
8     -0.009002 -0.000516       9        1   4.335724
12     0.014567 -0.040167      10        1   4.118653
1      0.018755 -0.003745      11        1   3.787209
17    -0.023436  0.021141      12        1   3.713186
15    -0.007276  0.013955      13        1   3.285484
6      0.054595  0.008982      14        1   3.114855
18     0.019690  0.014655      15        1   3.063771
5      0.019493  0.000298      16        1   2.651547
4     -0.016294 -0.018154      17        1   2.581069
3     -0.038528 -0.019674      18        1   2.093165
16    -0.002931  0.012879      19        1   1.564403
19     0.000219 -0.000887      20        1   1.183605, topic_info=    Term        Freq       Total Category  logprob  loglift
159   학문  352.000000  352.000000  Default  30.0000  30.0000
68    생각  413.000000  413.000000  Default  29.0000  29.0000
156   하니  336.000000  336.000000  Default  28.0000  28.0000
282   마음  312.000000  312.000000  Default  27.0000  27.0000
214   가지  230.000000  230.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
485   폐단    1.257541  103.191605  Topic20  -6.2417   0.0292
33    더욱    1.369652  159.282288  Topic20  -6.1563  -0.3195
38    때문    1.362247  160.999023  Topic20  -6.1618  -0.3357
560  일찍이    1.255449  105.371941  Topic20  -6.2434   0.0066
120   자가    1.268115  174.545074  Topic20  -6.2334  -0.4880

[1536 rows x 6 columns], token_table=      Topic      Freq Term
term                      
3495      6  0.312462   가선
3495     11  0.312462   가선
214       1  0.182096   가지
214       2  0.112726   가지
214       3  0.073705   가지
...     ...       ...  ...
7417     10  0.655438   흠앙
2369      1  0.499397   힐난
2369      3  0.124849   힐난
2369      8  0.124849   힐난
2369     17  0.124849   힐난

[5187 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 10, 11, 15, 14, 12, 1, 9, 13, 2, 18, 16, 7, 19, 6, 5, 4, 17, 20])

In [32]:
# 훈민정음

In [33]:
data =[]
for i in range(0,1):
    txt1 = open('세종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '세종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '훈민정음' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'세종실록'))

In [34]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("세종실록"," "))
df=df.map(lambda x : x.replace("세종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [35]:
len(df)

5

In [36]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	글자	0.01380189135670662
	사성	0.01357988454401493
	청탁	0.011915363371372223
	소리	0.010797451250255108
	서로	0.010478232055902481
	발성	0.00932352989912033
	중국	0.008835358545184135
	우리	0.008508249185979366
	모두	0.008166924118995667
	천지	0.008015775121748447


Topic ID: 1
	글자	0.011430841870605946
	소리	0.008144919760525227
	청탁	0.007416208274662495
	서로	0.006761885713785887
	사성	0.006116823293268681
	자모	0.0057509890757501125
	조화	0.005218996200710535
	어찌	0.005015186499804258
	모두	0.004595188423991203
	천지	0.004486386198550463


Topic ID: 2
	글자	0.019004058092832565
	소리	0.013901310972869396
	사성	0.011789730750024319
	청탁	0.009852772578597069
	우리	0.007455534301698208
	어찌	0.007423272356390953
	자모	0.0066005149856209755
	서로	0.006576940417289734
	성운	0.006250317674130201
	집현전	0.006136495620012283


Topic ID: 3
	발성	0.014185147359967232
	중성	0.006443666759878397
	가로	0.005954220425337553
	소리	0.005795205011963844
	수가	0.005751402582973242
	글자	0.005374421365559101
	모두	0.0039089643396437

In [37]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.115802 -0.030649       1        1  52.811836
11     0.148327 -0.087907       2        1  33.313770
13     0.111143  0.133321       3        1  11.310913
15    -0.025817  0.023706       4        1   2.090109
0     -0.015826 -0.021906       5        1   0.033940
6     -0.075286  0.026305       6        1   0.033845
14    -0.018592 -0.016948       7        1   0.033810
2     -0.023217 -0.012413       8        1   0.033657
3      0.024051 -0.009478       9        1   0.033021
10    -0.004555  0.015249      10        1   0.032472
8      0.010440  0.002932      11        1   0.032256
4      0.089650 -0.040137      12        1   0.028469
5     -0.073437 -0.006894      13        1   0.027849
16    -0.050254 -0.000118      14        1   0.027706
1     -0.021514  0.003849      15        1   0.027423
7      0.031584 -0.009060      16        1   0.027249
19    -0.004192  0.007722      17        1   0.026996
9      0.010985  0.004212      18        1   0.026790
17    -0.000417  0.008686      19        1   0.026749
18     0.002729  0.009526      20        1   0.021149, topic_info=    Term       Freq      Total Category  logprob  loglift
1     글자  23.000000  23.000000  Default   30.000  30.0000
68    발성  23.000000  23.000000  Default   29.000  29.0000
98    소리  18.000000  18.000000  Default   28.000  28.0000
327   사성  10.000000  10.000000  Default   27.000  27.0000
163   중성  10.000000  10.000000  Default   26.000  26.0000
..   ...        ...        ...      ...      ...      ...
164   지극   0.000413   2.207306  Topic20   -6.107  -0.1223
165   지취   0.000413   1.190187  Topic20   -6.107   0.4954
140   이개   0.000413   2.314845  Topic20   -6.107  -0.1699
139   응교   0.000413   2.067408  Topic20   -6.107  -0.0568
138   율려   0.000413   1.340452  Topic20   -6.107   0.3765

[1443 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
203       3  1.295925    가례
20        1  0.213841    가로
20        2  0.748444    가로
20        3  0.106921    가로
204       1  0.880941    가지
...     ...       ...   ...
448       1  0.838806    후생
192       2  0.970004    후음
19        1  0.250187  훈민정음
19        2  0.250187  훈민정음
19        3  0.375280  훈민정음

[371 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 12, 14, 16, 1, 7, 15, 3, 4, 11, 9, 5, 6, 17, 2, 8, 20, 10, 18, 19])

In [55]:
# 상소

In [56]:
data =[]
for i in range(0,1):
    txt1 = open('세종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '세종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '상소' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'세종실록'))

In [57]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("세종실록"," "))
df=df.map(lambda x : x.replace("세종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [58]:
len(df)

871

In [59]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]
# 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary

NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	백성	0.012859709560871124
	모두	0.010651341639459133
	상소	0.00940511841326952
	비록	0.00826723501086235
	어찌	0.007384949363768101
	생각	0.006119503173977137
	이제	0.005375197622925043
	다시	0.0050298855639994144
	자가	0.004950362723320723
	하니	0.004516602028161287


Topic ID: 1
	비록	0.010466154664754868
	모두	0.010273413732647896
	백성	0.009514762088656425
	생각	0.008381479419767857
	어찌	0.00818889494985342
	상소	0.0066802045330405235
	마음	0.006281586829572916
	하니	0.005818977486342192
	자가	0.004750458057969809
	반드시	0.004045036621391773


Topic ID: 2
	생각	0.009273865260183811
	상소	0.008922016248106956
	어찌	0.008553816936910152
	마음	0.00803957600146532
	이제	0.0073767248541116714
	백성	0.007052361499518156
	자가	0.006537092849612236
	비록	0.006438178010284901
	신하	0.0060989586636424065
	모두	0.005992450751364231


Topic ID: 3
	상소	0.0076537299901247025
	모두	0.00755296740680933
	태종	0.0069097005762159824
	국가	0.005271726753562689
	성상	0.0050946008414030075
	하니	0.0050894515588879585
	생각	0.004977142438

In [60]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.031143  0.003594       1        1  14.747124
18    -0.031644  0.007129       2        1  12.497561
2     -0.022559  0.011654       3        1  11.277077
0      0.004804 -0.022471       4        1   8.918725
11    -0.024363  0.009013       5        1   6.539788
17     0.008797 -0.013736       6        1   5.201729
1      0.000924 -0.002010       7        1   4.980261
16    -0.011919  0.003009       8        1   4.731855
14     0.006759 -0.003313       9        1   4.611710
4      0.001747  0.005428      10        1   4.420087
10     0.014980 -0.016317      11        1   3.924154
5     -0.029187  0.003099      12        1   3.474189
6     -0.020403 -0.003472      13        1   2.783547
12    -0.018188  0.007248      14        1   2.498451
8      0.025383 -0.002398      15        1   2.338133
13     0.009990 -0.044690      16        1   1.994785
15     0.008514 -0.011998      17        1   1.943475
3      0.031463  0.060456      18        1   1.296325
7      0.052052 -0.001136      19        1   1.114699
9      0.023994  0.010910      20        1   0.706326, topic_info=    Term         Freq        Total Category  logprob  loglift
46    백성   954.000000   954.000000  Default  30.0000  30.0000
40    모두   967.000000   967.000000  Default  29.0000  29.0000
53    상소  1243.000000  1243.000000  Default  28.0000  28.0000
158   비록   871.000000   871.000000  Default  27.0000  27.0000
69    어찌  1043.000000  1043.000000  Default  26.0000  26.0000
..   ...          ...          ...      ...      ...      ...
79    윤허     2.609310   517.498962  Topic20  -5.8228  -0.3371
214   가지     2.555879   504.576691  Topic20  -5.8435  -0.3325
107   폐단     2.291794   346.152496  Topic20  -5.9525  -0.0647
111  하물며     2.096655   282.119080  Topic20  -6.0415   0.0509
243   더욱     2.072794   305.287018  Topic20  -6.0530  -0.0395

[1592 rows x 6 columns], token_table=       Topic      Freq Term
term                       
10301      4  0.494127   가계
5978      10  0.204840   가묘
5978      12  0.204840   가묘
6749       1  0.139253   가식
6749       2  0.278506   가식
...      ...       ...  ...
5754      11  0.579682   흉포
7406       1  0.244391   흉풍
2166       6  0.378346   흠로
10383     13  0.733267   흥종
7115       6  0.596729   희천

[5492 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 19, 3, 1, 12, 18, 2, 17, 15, 5, 11, 6, 7, 13, 9, 14, 16, 4, 8, 10])